# XOR Deep Learning - Tensorflow 1.x

In [5]:
! pip uninstall tensorflow

Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.1


In [8]:
! pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 29kB/s 
     |████████████████████████████████| 3.8MB 46.0MB/s 
     |████████████████████████████████| 512kB 55.3MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=65e24a60a52dcde7aeba5681cd20e0c739434e7313b53d779cb833c7ab8075f3
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [1]:
import tensorflow as tf

print(tf.__version__)

1.15.0


In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report

# Training Data Set
x_data = np.array([[0,0], [0,1], [1,0], [1,1]], dtype=np.float32)
t_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

# placeholder (input layer역할)
X = tf.placeholder(shape=[None, 2], dtype=tf.float32) # 4행 2열
T = tf.placeholder(shape=[None, 1], dtype=tf.float32) # 4행 1열

# 2번째 layer (hidden layer)
W2 = tf.Variable(tf.random.normal([2, 10])) # 그 다음 hidden layer로 넘어감
b2 = tf.Variable(tf.random.normal([10])) # 출력이 10개 나감 # 2번째 hidden layer가 다중분류가 됨
layer2 = tf.sigmoid(tf.matmul(X,W2) + b2) # 4행 10열, 그 다음 레이어의 input으로 들어감

# 3번째 layer (hidden layer)
W3 = tf.Variable(tf.random.normal([10, 6])) 
b3 = tf.Variable(tf.random.normal([6])) 
layer3 = tf.sigmoid(tf.matmul(layer2,W3) + b3)

# output layer
W4 = tf.Variable(tf.random.normal([6, 1])) 
b4 = tf.Variable(tf.random.normal([1]))

# Hypothesis
logit = tf.matmul(layer3, W4) + b4 # (입력, 가중치)
H = tf.sigmoid(logit)

# loss
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-2).minimize(loss)

# learning
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(30000):
  _, loss_val = sess.run([train, loss], feed_dict={X: x_data, T: t_data})

  if step % 3000 == 0:
    print('loss: {}'.format(loss_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
loss: 0.9111691117286682
loss: 0.6963692307472229
loss: 0.6907936334609985
loss: 0.6852415800094604
loss: 0.676274299621582
loss: 0.6586703658103943
loss: 0.6205938458442688
loss: 0.5399441719055176
loss: 0.3983103632926941
loss: 0.22943688929080963


In [4]:
accuracy = tf.cast(H >= 0.5, dtype=tf.float32)
result = sess.run(accuracy, feed_dict={X: x_data})
print(classification_report(t_data.ravel(), result.ravel())) # accuracy: 1

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         2
         1.0       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

